# **LIBRARIES**

In [107]:
import collections
import helper
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# **LOAD DATA**

In [108]:
import os

def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()
    return data.split('\n')
def load_data_array(path,index):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()
        array = []
        for sentence in data:
          result = sentence.split(';')
          array.append(result[index])
    return array

In [149]:
# english_sentences = load_data('es.txt')
# french_sentences = load_data('nasa.txt')
english_sentences = load_data('en.txt')
french_sentences = load_data('fr.txt')
print('Dataset Loaded')

Dataset Loaded


In [150]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


# **VOCABULARY**

In [151]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


# **TOKENIZE**

In [152]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


# **PADDING**

In [153]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


# **PRE PROCESS**

In [154]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


# **IDS BACK TO TEXT**

In [115]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


# **RNN**

In [121]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 6.7923 - accuracy: 0.5941 - val_loss: nan - val_accuracy: 0.5656
Epoch 2/10
1/1 [==============================] - 0s 49ms/step - loss: 6.7872 - accuracy: 0.5941 - val_loss: nan - val_accuracy: 0.5674
Epoch 3/10
1/1 [==============================] - 0s 47ms/step - loss: 6.7820 - accuracy: 0.6017 - val_loss: nan - val_accuracy: 0.5709
Epoch 4/10
1/1 [==============================] - 0s 32ms/step - loss: 6.7766 - accuracy: 0.6097 - val_loss: nan - val_accuracy: 0.5727
Epoch 5/10
1/1 [==============================] - 0s 47ms/step - loss: 6.7711 - accuracy: 0.6128 - val_loss: nan - val_accuracy: 0.5762
Epoch 6/10
1/1 [==============================] - 0s 49ms/step - loss: 6.7654 - accuracy: 0.6142 - val_loss: nan - val_accuracy: 0.5798
Epoch 7/10
1/1 [==============================] - 0s 30ms/step - loss: 6.7595 - accuracy: 0.6142 - val_loss: nan - val_accuracy: 0.5798
Epoch 8/10
1/1 [==============================] - 

# **EMBEDDING**

In [122]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 6.7990 - accuracy: 4.4504e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 36ms/step - loss: 6.7924 - accuracy: 4.4504e-04 - val_loss: nan - val_accuracy: 0.0053
Epoch 3/10
1/1 [==============================] - 0s 33ms/step - loss: 6.7858 - accuracy: 0.0116 - val_loss: nan - val_accuracy: 0.5762
Epoch 4/10
1/1 [==============================] - 0s 32ms/step - loss: 6.7791 - accuracy: 0.6182 - val_loss: nan - val_accuracy: 0.5833
Epoch 5/10
1/1 [==============================] - 0s 36ms/step - loss: 6.7722 - accuracy: 0.6190 - val_loss: nan - val_accuracy: 0.5833
Epoch 6/10
1/1 [==============================] - 0s 38ms/step - loss: 6.7650 - accuracy: 0.6195 - val_loss: nan - val_accuracy: 0.5833
Epoch 7/10
1/1 [==============================] - 0s 34ms/step - loss: 6.7573 - accuracy: 0.6195 - val_loss: nan - val_accuracy: 0.5833
Epoch 8/10
1/1 [======================

# **BIDIRECTIONAL RNN**

In [123]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 6.7989 - accuracy: 0.5968 - val_loss: 6.7868 - val_accuracy: 0.5656
Epoch 2/20
1/1 [==============================] - 0s 36ms/step - loss: 6.7857 - accuracy: 0.5950 - val_loss: 6.7762 - val_accuracy: 0.5674
Epoch 3/20
1/1 [==============================] - 0s 36ms/step - loss: 6.7714 - accuracy: 0.5986 - val_loss: 6.7655 - val_accuracy: 0.5691
Epoch 4/20
1/1 [==============================] - 0s 34ms/step - loss: 6.7578 - accuracy: 0.6075 - val_loss: 6.7544 - val_accuracy: 0.5816
Epoch 5/20
1/1 [==============================] - 0s 36ms/step - loss: 6.7425 - accuracy: 0.6159 - val_loss: 6.7430 - val_accuracy: 0.5816
Epoch 6/20
1/1 [==============================] - 0s 35ms/step - loss: 6.7286 - accuracy: 0.6164 - val_loss: 6.7312 - val_accuracy: 0.5816
Epoch 7/20
1/1 [==============================] - 0s 34ms/step - loss: 6.7127 - accuracy: 0.6168 - val_loss: 6.7188 - val_accuracy: 0.5816
Epoch 8/20
1/1 [=============

# **ENCODER DECODER**

In [124]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 6.7941 - accuracy: 0.0000e+00 - val_loss: 6.7347 - val_accuracy: 0.5691
Epoch 2/20
1/1 [==============================] - 0s 40ms/step - loss: 6.7295 - accuracy: 0.6119 - val_loss: 6.6682 - val_accuracy: 0.5762
Epoch 3/20
1/1 [==============================] - 0s 36ms/step - loss: 6.6578 - accuracy: 0.6182 - val_loss: 6.5880 - val_accuracy: 0.5833
Epoch 4/20
1/1 [==============================] - 0s 42ms/step - loss: 6.5712 - accuracy: 0.6195 - val_loss: 6.4862 - val_accuracy: 0.5833
Epoch 5/20
1/1 [==============================] - 0s 39ms/step - loss: 6.4614 - accuracy: 0.6195 - val_loss: 6.3540 - val_accuracy: 0.5833
Epoch 6/20
1/1 [==============================] - 0s 37ms/step - loss: 6.3190 - accuracy: 0.6195 - val_loss: 6.1812 - val_accuracy: 0.5833
Epoch 7/20
1/1 [==============================] - 0s 35ms/step - loss: 6.1334 - accuracy: 0.6195 - val_loss: 5.9562 - val_accuracy: 0.5833
Epoch 8/20
1/1 [=========

# **CUSTOM**

In [155]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(512,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(512,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

print('Final Model Loaded')

Final Model Loaded


# **PREDICTION**

In [157]:
def final_predictions(x, y, x_tk, y_tk):
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 20, validation_split = 0.2)
 
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    sentence = 'we like oranges mangoes and grapes '
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))
    print('Result 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    # print('Result 2:')
    # print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    # print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/20
108/108 [==============================] - 35s 271ms/step - loss: 3.7019 - accuracy: 0.4300 - val_loss: 2.9008 - val_accuracy: 0.4753
Epoch 2/20
108/108 [==============================] - 30s 281ms/step - loss: 2.7780 - accuracy: 0.4905 - val_loss: 2.5923 - val_accuracy: 0.5363
Epoch 3/20
108/108 [==============================] - 29s 271ms/step - loss: 2.5381 - accuracy: 0.5439 - val_loss: 2.4311 - val_accuracy: 0.5720
Epoch 4/20
108/108 [==============================] - 29s 269ms/step - loss: 2.3876 - accuracy: 0.5794 - val_loss: 2.4237 - val_accuracy: 0.5613
Epoch 5/20
108/108 [==============================] - 30s 276ms/step - loss: 2.2847 - accuracy: 0.6002 - val_loss: 2.2254 - val_accuracy: 0.6035
Epoch 6/20
108/108 [==============================] - 30s 274ms/step - loss: 2.1709 - accuracy: 0.6222 - val_loss: 2.0787 - val_accuracy: 0.6603
Epoch 7/20
108/108 [==============================] - 29s 273ms/step - loss: 2.0756 - accuracy: 0.6457 - val_loss: 2.0468 - val_ac